# Create Table of labels

In [ ]:
whoami

In [1]:
# Main Imports

import nilearn

from nilearn.image import resample_to_img

import pylab as plt

import numpy as np
import nibabel as nb
import os
import glob
import random
import pandas as pd
import re

import seaborn as sns #added
sns.set(style="darkgrid") #added

from nilearn.image import mean_img #added
from nilearn.plotting import plot_anat #added

ModuleNotFoundError: No module named 'nilearn'

# Principal code parameters

In [57]:
ls {rootDirectory}/{dataDir}/*csv

/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/Array_Labels_Def.csv*
/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/EGFR_Labels.csv*
/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/IDH_Labels.csv*
/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/KI67_Labels.csv*
/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/MGMT_Labels.csv*


In [67]:
# Local directory where the Data are mounted
rootDirectory = "/data/RMN/LUCA_PASQUINI"

# Local subdirectory where dataset is mounted
dataDir = "DATI_SEGMENTATI_SCALATI_media"

# Dataset dir
datasetDir = f"{rootDirectory}/{dataDir}"

#CSV fileroot 
fileName = f"{rootDirectory}/{dataDir}/KI67_Labels.csv" # Array_Labels_Def.csv"

SUBJECT_NAME_REPEATED=["BIANCHI","BOVE","PROIETTI"]

PATIENT_REMOVED=['Pascal','Mitchell','Rufini','Farella','Array']

SEQUENCE_1= "FLAIR_registered"

#SEQUENCE_2= "rCBV"
#T1_registered , T1
#T2_registered , T2
#FLAIR_registered , FLAIR
#ADC_registered , ADC
#rCBV_registered , rCBV
#MPRAGEMDC

MaskPath = "T2ROI"

#SOLID
#NECROSI
#T1ROI
#T2ROI

# Survival Labels dataframe

In [68]:
#CSV fileroot 
#fileName = "/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/Array_Labels_Def.csv"

#csv file
df = pd.read_csv(fileName, sep=";", header=None)

df = df.drop([2], axis=1)

#Dataframe columns title
df.columns = ["Subject", "Survival"]

#Subject column as index
df = df.set_index('Subject')

#in order to check the entire dataframe of labels
pd.set_option('display.max_rows', len(df))
print(df)

                Survival
Subject                 
Alessandrini         0.0
Angeloni             1.0
Assanto              1.0
Bagnoli              0.0
Barontini            1.0
Battista             1.0
Bernola              0.0
Bertuzzi             0.0
Bevilacqua           1.0
BianchiG             1.0
BianchiO             1.0
Boezi                0.0
Cacace               1.0
Camacci              0.0
Camplese             1.0
Capezzone            1.0
Carzedda             1.0
Cellini              0.0
Cherri               0.0
Colafrancesco        1.0
Colamartini          1.0
Colazzo              1.0
Coletta              1.0
Cosimi               1.0
Crescenzi            1.0
BoveP                1.0
Angeli               1.0
Diaspro              0.0
Carlantonio          0.0
Lorenzo              1.0
Massa                1.0
Dobrisan             1.0
Droghei              1.0
Angelo               0.0
Darida               1.0
Santis               1.0
Maso                 1.0
Fabiani              1.0


# Functions

In [69]:
#NORMALIZE_NAME: the function takes the subject path basename and splits it according to the underscore
#in order to take the first string and use it as index in dataframe.
#If a string is repeated, the function adds the first letter of the second string after the underscore

def normalize_name(subject_path, add_name):
    #Components= []
    subject = os.path.basename(subject_path)
    subject = re.sub(r'^(DE|D|DI|LO|DEL)_', '', subject)
    components =  subject.split("_")
    if add_name[0]==components[0] or add_name[1]==components[0] or add_name[2]==components[0] :
            return components[0].title() + components[1][0].capitalize()
    else:
            return components[0].title()
        
    
#GET_SUBJECT_METADATA:the function recives, as input, the subject path and the list of name repeated already known
#the output are the path base name (SURNAME_NAME) and the normalized name
#add_name needs to contain the list of name repeated


def get_subject_metadata(subject_path, subjects_with_name=[]):
    dirname = os.path.basename(subject_path)
    return (dirname, normalize_name(subject_path, add_name=[name for name in subjects_with_name]))

In [70]:
'''' Get index positions of value in dataframe '''
def getIndexes(dfObj, value):
    listOfPos = list()
    result = dfObj.isin([value]) # Get bool dataframe with True at positions where the given value exists
    seriesObj = result.any() # Get list of columns that contains the value
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames: # Iterate over list of columns and fetch the rows indexes where value exists
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
# Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

# Dataframe folders construction

In [71]:
#Path and Subject columns
Df_Subjects_dirs = pd.DataFrame()
for subject_path in glob.glob(f"{datasetDir}/*"):
    subjects_dirs=[get_subject_metadata(subject_path , subjects_with_name=[SUBJECT_NAME for SUBJECT_NAME in SUBJECT_NAME_REPEATED])]
    print(subjects_dirs)
    df_subject_dirs = pd.DataFrame(subjects_dirs,columns=["Path", "Subject"])
    Df_Subjects_dirs=pd.concat([Df_Subjects_dirs,df_subject_dirs],ignore_index=True)


#in order to check the correct constucrion of 
pd.set_option('display.max_rows', len(Df_Subjects_dirs))
print(Df_Subjects_dirs)

#Subject as index
Df_Subjects_dirs = Df_Subjects_dirs.set_index('Subject')
pd.set_option('display.max_rows', len(Df_Subjects_dirs))

#in order to check the correct constucrion of Df_Subjects_dirs
print(Df_Subjects_dirs)

#Unnecessary strings removal with "drop" function
x=Df_Subjects_dirs
x.drop(index='Array', columns='Path')
# Delete rows with index label a & b    
modX= x.drop([PATIENT for PATIENT in PATIENT_REMOVED])
#modX["Path"]
modX = pd.DataFrame(modX)

#in order to check the correct construction of the dataframe
pd.set_option('display.max_rows', len(modX))
print(modX)

[('ALESSANDRINI_GLAUCO', 'Alessandrini')]
[('ANGELONI_GIUSEPPINA', 'Angeloni')]
[('Array_Labels_Def.csv', 'Array')]
[('ASSANTO_MARIA', 'Assanto')]
[('ASTA_G', 'Asta')]
[('BAGNOLI_VINCENZO', 'Bagnoli')]
[('BALZAMO_A', 'Balzamo')]
[('BARONTINI_MARIA_GIOVANNA', 'Barontini')]
[('BATTISTA_DOMENICA', 'Battista')]
[('BERGNACH_SILVANO', 'Bergnach')]
[('BERNOLA_TERESA', 'Bernola')]
[('BERTUZZI_LUISA', 'Bertuzzi')]
[('BEVILACQUA_RITA', 'Bevilacqua')]
[('BIANCHI_GIOVANNI', 'BianchiG')]
[('BIANCHI_ORAZIO', 'BianchiO')]
[('BIANCHI_S', 'BianchiS')]
[('BIAVATI_S', 'Biavati')]
[('BOEZI_MARIO', 'Boezi')]
[('BOVE_A', 'BoveA')]
[('CACACE_PAOLO', 'Cacace')]
[('CALDARONI_ANNA', 'Caldaroni')]
[('CAMACCI_FILIBERTO', 'Camacci')]
[('CAMPLESE_CANDEROLA', 'Camplese')]
[('CAPEZZONE', 'Capezzone')]
[('CARULLI_L', 'Carulli')]
[('CARZEDDA_PAOLO', 'Carzedda')]
[('CATALANI_F', 'Catalani')]
[('CELLINI_T', 'Cellini')]
[('CHERRI_M', 'Cherri')]
[('CIMPUREANU_N', 'Cimpureanu')]
[('COLAFRANCESCO_ROCCO', 'Colafrancesco')]
[(

# Join function between Path and Labels

In [72]:
result = modX.join(df, on='Subject')

#in order to check if all the dataframe is construced in the right way
pd.set_option('display.max_rows', len(result))
result   

,Path,Survival
Subject,,
Alessandrini,ALESSANDRINI_GLAUCO,0.0
Angeloni,ANGELONI_GIUSEPPINA,1.0
Assanto,ASSANTO_MARIA,1.0
Asta,ASTA_G,NaN
Bagnoli,BAGNOLI_VINCENZO,0.0
Balzamo,BALZAMO_A,NaN
Barontini,BARONTINI_MARIA_GIOVANNA,1.0
Battista,BATTISTA_DOMENICA,1.0
Bergnach,BERGNACH_SILVANO,NaN


# NAN removal

In [73]:
result = result.dropna()

In [74]:
"""
#it takes the indexes to which the NaN corresponds
NAN_index=result['Survival'].index[result['Survival'].apply(np.isnan)]

#it takes the number of the corresponding row as int
df_index=result.index.values.tolist()
int_index=[df_index.index(i) for i in NAN_index]
#int_index

#than it is possible to obtain the corresponding survival label of the original array, df in this case
label=[df.iloc[x]['Survival'] for x in int_index]
label

#iteration to replace the NaNs
for l in label:
    result['Survival'].fillna(l,inplace=True)
    
#result is the final Dataframe with "Subject" as index and the columns "Path" and "Survival"
pd.set_option('display.max_rows', len(result))
result
"""

'\n#it takes the indexes to which the NaN corresponds\nNAN_index=result[\'Survival\'].index[result[\'Survival\'].apply(np.isnan)]\n\n#it takes the number of the corresponding row as int\ndf_index=result.index.values.tolist()\nint_index=[df_index.index(i) for i in NAN_index]\n#int_index\n\n#than it is possible to obtain the corresponding survival label of the original array, df in this case\nlabel=[df.iloc[x][\'Survival\'] for x in int_index]\nlabel\n\n#iteration to replace the NaNs\nfor l in label:\n    result[\'Survival\'].fillna(l,inplace=True)\n    \n#result is the final Dataframe with "Subject" as index and the columns "Path" and "Survival"\npd.set_option(\'display.max_rows\', len(result))\nresult\n'

In [75]:
result.to_csv("dataset-ki67.csv")